# Downloading, and creating a dataframe from NHTS data


In [1]:
import os
import wget
import ssl
import zipfile
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

In [2]:
# To address verfication error while dowloading the data
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:

if os.path.exists('./data/Ascii.zip'):
     pass
else:
     if os.path.exists('./data/'):
          pass
     else:
          os.mkdir('./data/')
     # Download the survey data
     url = 'https://nhts.ornl.gov/2009/download/Ascii.zip'
     wget.download(url, './data/Ascii.zip')



In [4]:
with zipfile.ZipFile("./data/Ascii.zip","r") as zip_ref:
    zip_ref.extractall("./data/")
    
Dir = os.path.join('./data/Ascii')
showlist = os.listdir(Dir)

# Observe the folder's content
print(showlist)

['DAYV2PUB.CSV', 'HHV2PUB.CSV', 'PERV2PUB.CSV', 'VEHV2PUB.CSV']


After the zip file is downloaded, we'll unzip and extract its content into our directory.The unpacked file is consist of four .csv files as follows:

- **Travel day trips**
- Household Data 
- **Person Data**
- Vehicle Data 





In [5]:
trp_dir = ('./data/Ascii/DAYV2PUB.CSV')
prsn_dir = ('./data/Ascii/PERV2PUB.CSV')

# df is Travel day trip dataframe
df = pd.read_csv(trp_dir)

# df_p is each person's dataframe
df_p = pd.read_csv(prsn_dir)

# Preprocess; Reshaping the data as we need it

## National Houshold Travel Survey

In [6]:
# the lists of variables we wish to keep
df = df[
    ['HOUSEID', 'PERSONID','WHYFROM', 'WHYTO', 'TRPMILES',
     'HHSTATE', 'STRTTIME','ENDTIME','TDTRPNUM', 'DWELTIME',
     'URBRUR', 'TRVL_MIN','HBHUR','TRIPPURP', 'R_SEX', 'R_AGE',
     'HHFAMINC', 'TRPTRANS'
    ]
]
df_p = df_p[
    ['HOUSEID', 'PERSONID', 'R_SEX', 'R_AGE', 'HHFAMINC','HHSIZE',
     'HHVEHCNT', 'HHSTATE', 'URBRUR', 'SAMEPLC', 'HBHUR', 'TRAVDAY',
     'HH_RACE', 'DRIVER', 'EDUC', 'OCCAT', 'SELF_EMP', 'WKFTPT', 'FRSTHM'
    ]
]

# -1 are not nan values in this column.                   
df_p['SAMEPLC'].replace([-1, -7], 0, inplace=True)


## Replacing values in columns 

In [7]:
# Compacting trip modes to 5 main categories
df['TRPTRANS'].replace([1, 2, 3, 4, 5, 6, 7, 8], 2, inplace=True) # auto
df['TRPTRANS'].replace([9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 3, inplace=True) # transit
df['TRPTRANS'].replace([23], 4, inplace=True) # walk
df['TRPTRANS'].replace([22], 5, inplace=True) # bike
df['TRPTRANS'].replace([97, 20, 21, 24], 6, inplace=True) # other

df['TRPTRANS'].replace([-1, -7, -8, -9], np.nan, inplace=True)
df['TRPTRANS'].value_counts()

2.0    1021207
4.0     100405
3.0      26727
5.0       9443
6.0       7415
Name: TRPTRANS, dtype: int64

### Three new columns are defined to indicate the number of trips specifically seperated by purpose:

Trips in the NHTS data are divided into detailed trip purposes. For the purpose of the paper, all these categories will be grouped into 3 main purposes: 1-work and education trips 2- shop trips 3- other trips 

In [8]:
# Change the items in the given list with
work_education = [10, 11, 12] # = 2
shop_trips = [40, 41, 42] # = 3
other_trips = [20, 21, 22, 23, 24, 30, 43, 60,
               61, 62, 63, 64, 65, 97, 80, 81,
               82, 83, 70, 71, 72, 73, 50, 51,
               52, 53, 54, 55, 1, 13, 14] # = 4

# Regrouping purposes
df['WHYTO'].replace(work_education, 2, inplace=True)
df['WHYTO'].replace(shop_trips, 3, inplace=True)
df['WHYTO'].replace(other_trips, 4, inplace=True)

# Replacing not available data with NAN
df['WHYTO'].replace([-1, -7, -8, -9], np.nan, inplace=True)
df['STRTTIME'].replace([-1, -7, -8, -9], np.nan, inplace=True)
df['TRIPPURP'].replace('-9', np.nan, inplace=True)


#changing not available data for these two to 0 because we don't want to delete them
df['DWELTIME'].replace([-1, -7, -8, -9], 0, inplace=True)
df['TRPMILES'].replace([-1, -7, -8, -9], 0, inplace=True)

value_counts = df['WHYTO'].value_counts().values
print('Total value counts for education-work(mandatory) trips is: {}'.format(value_counts[2]))
print('Total value counts for shopping trips is {}'.format(value_counts[1]))
print('Total value counts for other trips is: {}'.format(value_counts[0]))

Total value counts for education-work(mandatory) trips is: 92152
Total value counts for shopping trips is 209607
Total value counts for other trips is: 859952


In [9]:
# defining 3 columns that shows the number of trips by category
counts = df.groupby(["HOUSEID", "PERSONID", "WHYTO"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"WHYTO{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'WHYTO2.0': 'WORK_EDUCATION', 'WHYTO3.0': 'SHOP', 'WHYTO4.0': 'OTHER'}, inplace=True)

### Home based and non-homebased trips

Home based trips in the data, are categorized as homebased shop, homebased work, homebased social-recreational, homebased other and non-homebased. I added the homebased social-recreational to homebased other to shrink the range.

In [10]:
df['TRIPPURP'].unique()

array(['HBO', 'NHB', 'HBSOCREC', 'HBSHOP', 'HBW', nan], dtype=object)

In [11]:
# Recreational trips are in the other category
df['TRIPPURP'].replace('HBSOCREC', 'HBO', inplace=True)

In [12]:
# Changing gender classification categories to 0(male) and 1(female)
df_p['R_SEX'].replace(1, 0, inplace=True)
df_p['R_SEX'].replace(2, 1, inplace=True)

### Defining time intervals to temporally count the trips

4 periods of day are defined to see how many trips are made in each period. These periods are each a column that will be added to the features later.

In [13]:
# 1 = A.M
# 2 = P.M
# 3 = mid_day
# 4 = night

df['TRVL_DAY_TIME'] = np.where((df['STRTTIME'] >= 500) & (df['STRTTIME'] <= 1000), 1, 
                  np.where((df['STRTTIME'] >= 1001) & (df['STRTTIME'] <= 1530), 2,
                  np.where((df['STRTTIME'] >= 1531) & (df['STRTTIME'] <= 2000), 3, 4)))

In [14]:
# making the columns for each daytime and merging it to the person dataframe
counts1 = df.groupby(["HOUSEID", "PERSONID", "TRVL_DAY_TIME"]).apply(len).unstack(fill_value=0)
counts1.columns = counts1.columns.map(lambda x: f"STRTTIME{x}")
df_p = df_p.merge(counts1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'STRTTIME1': 'AM', 'STRTTIME2': 'PM', 'STRTTIME3': 'MIDDAY', 'STRTTIME4': 'NIGHT'}, inplace=True)

The data was collected on a travel day. travel day is either a day on weekday or the weekend. To be able to process this later and analyse the difference, lets do the following:

In [15]:
# the data was collected based on trips on weekday or weekend?
df_p['TRAVDAY'].replace([1, 7], 'weekend', inplace=True)
df_p['TRAVDAY'].replace([2, 3, 4, 5, 6], 'weekday', inplace=True)

###  Time of the first and the last trip

In [16]:
# This two new columns will show the time the person did their first and last trips as indicated in the name
res =(df.sort_values(["HOUSEID","PERSONID","TDTRPNUM"])
    .groupby(["HOUSEID", "PERSONID"], as_index=False)
    .agg({"STRTTIME":"first","ENDTIME":"last"})
    .rename(columns={"STRTTIME":"firsttrip_time","ENDTIME":"lasttrip_time"}))

In [17]:
# the time is stored in military time format.
# this will convert the number to integer numbers in range of 0 to 23
res['firsttrip_time'] = np.where((res['firsttrip_time'] >= 0) & (res['firsttrip_time'] <= 100), 0, 
                  np.where((res['firsttrip_time'] >= 101) & (res['firsttrip_time'] <= 200), 1,
                  np.where((res['firsttrip_time'] >= 201) & (res['firsttrip_time'] <= 300), 2,
                  np.where((res['firsttrip_time'] >= 301) & (res['firsttrip_time'] <= 400), 3,
                  np.where((res['firsttrip_time'] >= 401) & (res['firsttrip_time'] <= 500), 4,
                  np.where((res['firsttrip_time'] >= 501) & (res['firsttrip_time'] <= 600), 5,
                  np.where((res['firsttrip_time'] >= 601) & (res['firsttrip_time'] <= 700), 6,
                  np.where((res['firsttrip_time'] >= 701) & (res['firsttrip_time'] <= 800), 7,
                  np.where((res['firsttrip_time'] >= 801) & (res['firsttrip_time'] <= 900), 8,
                  np.where((res['firsttrip_time'] >= 901) & (res['firsttrip_time'] <= 1000), 9,
                  np.where((res['firsttrip_time'] >= 1001) & (res['firsttrip_time'] <= 1100), 10,
                  np.where((res['firsttrip_time'] >= 1101) & (res['firsttrip_time'] <= 1200), 11,
                  np.where((res['firsttrip_time'] >= 1201) & (res['firsttrip_time'] <= 1300), 12, 
                  np.where((res['firsttrip_time'] >= 1301) & (res['firsttrip_time'] <= 1400), 13,
                  np.where((res['firsttrip_time'] >= 1401) & (res['firsttrip_time'] <= 1500), 14,
                  np.where((res['firsttrip_time'] >= 1501) & (res['firsttrip_time'] <= 1600), 15,
                  np.where((res['firsttrip_time'] >= 1601) & (res['firsttrip_time'] <= 1700), 16,
                  np.where((res['firsttrip_time'] >= 1701) & (res['firsttrip_time'] <= 1800), 17,
                  np.where((res['firsttrip_time'] >= 1801) & (res['firsttrip_time'] <= 1900), 18,
                  np.where((res['firsttrip_time'] >= 1901) & (res['firsttrip_time'] <= 2000), 19,
                  np.where((res['firsttrip_time'] >= 2001) & (res['firsttrip_time'] <= 2100), 20,
                  np.where((res['firsttrip_time'] >= 2101) & (res['firsttrip_time'] <= 2200), 21,
                  np.where((res['firsttrip_time'] >= 2201) & (res['firsttrip_time'] <= 2300), 22,
                  np.where((res['firsttrip_time'] >= 2301) & (res['firsttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))

res['lasttrip_time'] = np.where((res['lasttrip_time'] >= 0) & (res['lasttrip_time'] <= 100), 0, 
                  np.where((res['lasttrip_time'] >= 101) & (res['lasttrip_time'] <= 200), 1,
                  np.where((res['lasttrip_time'] >= 201) & (res['lasttrip_time'] <= 300), 2,
                  np.where((res['lasttrip_time'] >= 301) & (res['lasttrip_time'] <= 400), 3,
                  np.where((res['lasttrip_time'] >= 401) & (res['lasttrip_time'] <= 500), 4,
                  np.where((res['lasttrip_time'] >= 501) & (res['lasttrip_time'] <= 600), 5,
                  np.where((res['lasttrip_time'] >= 601) & (res['lasttrip_time'] <= 700), 6,
                  np.where((res['lasttrip_time'] >= 701) & (res['lasttrip_time'] <= 800), 7,
                  np.where((res['lasttrip_time'] >= 801) & (res['lasttrip_time'] <= 900), 8,
                  np.where((res['lasttrip_time'] >= 901) & (res['lasttrip_time'] <= 1000), 9,
                  np.where((res['lasttrip_time'] >= 1001) & (res['lasttrip_time'] <= 1100), 10,
                  np.where((res['lasttrip_time'] >= 1101) & (res['lasttrip_time'] <= 1200), 11,
                  np.where((res['lasttrip_time'] >= 1201) & (res['lasttrip_time'] <= 1300), 12, 
                  np.where((res['lasttrip_time'] >= 1301) & (res['lasttrip_time'] <= 1400), 13,
                  np.where((res['lasttrip_time'] >= 1401) & (res['lasttrip_time'] <= 1500), 14,
                  np.where((res['lasttrip_time'] >= 1501) & (res['lasttrip_time'] <= 1600), 15,
                  np.where((res['lasttrip_time'] >= 1601) & (res['lasttrip_time'] <= 1700), 16,
                  np.where((res['lasttrip_time'] >= 1701) & (res['lasttrip_time'] <= 1800), 17,
                  np.where((res['lasttrip_time'] >= 1801) & (res['lasttrip_time'] <= 1900), 18,
                  np.where((res['lasttrip_time'] >= 1901) & (res['lasttrip_time'] <= 2000), 19,
                  np.where((res['lasttrip_time'] >= 2001) & (res['lasttrip_time'] <= 2100), 20,
                  np.where((res['lasttrip_time'] >= 2101) & (res['lasttrip_time'] <= 2200), 21,
                  np.where((res['lasttrip_time'] >= 2201) & (res['lasttrip_time'] <= 2300), 22,
                  np.where((res['lasttrip_time'] >= 2301) & (res['lasttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))
x = res.groupby(['HOUSEID','PERSONID']).sum()# this has no effect and it is done to just prevent an error
df_p = df_p.merge(x, how="left", left_on=['HOUSEID', 'PERSONID'], right_index=True)


### Removing outliers from the data

In [18]:
# trips longer than 2 hours and longer than 30 miles are removed in order to prevent bias
df = df[df['TRVL_MIN'] < 120]

# so are the trips longer than 30 miles
df = df[df['TRPMILES'] < 30]

# and also the trips that have no trips after them for more than 12 hours(720 minutes)
df = df[df['DWELTIME'] < 720]

In [ ]:
df_p.head()

### Dweltime, travel time, and travel distance of work and shop trips

In [20]:
# This makes 2 columns of mean travel time for work and shop trips 
new_df1=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRVL_MIN')
new_df1.fillna(0, inplace=True)

df_p = df_p.merge(new_df1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_traveltime', 3: 'shop_traveltime'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)


In [21]:
# Make a copy of distance column unchanged
df['TRPMILES2'] = df['TRPMILES'].copy()

In [22]:
# Sum of distance traveled by the person
new_df2=df.groupby(['HOUSEID','PERSONID'] )['TRPMILES'].sum()
df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_sum'}, inplace=True)

# Sum of the travel time of the person
new_df3=df.groupby(['HOUSEID','PERSONID'] )['TRVL_MIN'].sum()
df_p = df_p.merge(new_df3, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_sum'}, inplace=True)


#### Normalizing trip distances due to the residence state

Each state varies from others in terms of infrustructer development, urban or rural, state enormity, etc. thus the travel distance is normalized as follow to alleviate this problem and establish integrity in the data.

In [23]:
# Each trip is normalized by the max value of trip in that state
df["max_trpmiles"] = df.groupby("URBRUR")["TRPMILES"].transform("max")
df["TRPMILES"] /= df["max_trpmiles"]
df = df.drop("max_trpmiles", axis=1)

In [24]:
new_df=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRPMILES')
new_df.fillna(0, inplace=True)

df_p = df_p.merge(new_df, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_tripmile', 3: 'shop_tripmile'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)

In [25]:
# trip miles for each person, normalized by state and urban-rural classification
s=df.groupby(['HOUSEID',  'PERSONID'])['TRPMILES'].mean()
df_p = df_p.merge(s, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_mean'}, inplace=True)

#mean traveltime of all trips
s1=df.groupby(['HOUSEID',  'PERSONID'])['TRVL_MIN'].mean()
df_p = df_p.merge(s1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_mean'}, inplace=True)

#mean of the time the person is not traveling
s2=df.groupby(['HOUSEID',  'PERSONID'])['DWELTIME'].mean()
df_p = df_p.merge(s2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'DWELTIME': 'DWELTIME_mean'}, inplace=True)

In [26]:
# Mean time spent for shopping and work 
new_df2=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='DWELTIME')
new_df2.fillna(0, inplace=True)

df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_dweltime', 3: 'shop_dweltime'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)

In [27]:
# Counting the number of home based and non home based trips
counts = df.groupby(["HOUSEID", "PERSONID", "TRIPPURP"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.fillna(0, inplace=True)

In [28]:
# defining 3 columns that shows the number of trips by category
counts5 = df.groupby(["HOUSEID", "PERSONID", "TRPTRANS"]).apply(len).unstack(fill_value=0)
counts5.columns = counts5.columns.map(lambda x: f"TRPTRANS{x}")
df_p = df_p.merge(counts5, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPTRANS2.0': 'auto','TRPTRANS3.0': 'transit', 'TRPTRANS4.0': 'walk',
                     'TRPTRANS5.0': 'bike', 'TRPTRANS6.0': 'other'}, inplace=True)

In [29]:
# This makes a colum with calculated speed for each trip in miles/hour.(distance/travel time)
df['TRAVEL_SPEED'] = df['TRPMILES2']/df['TRVL_MIN']
df['TRAVEL_SPEED'] *= 60

In [30]:
# make average speed for each mode
new_df22=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='TRPTRANS',values='TRAVEL_SPEED')
new_df22.fillna(0, inplace=True)

df_p = df_p.merge(new_df22, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'auto_speed'}, inplace=True)
df_p.drop([4, 3, 5, 6], axis=1, inplace=True)

In [31]:
# setting under 18 people as np.nan

list_under18 = [i for i in range(0,18)]

df_p['R_AGE'].replace(list_under18, np.nan, inplace=True)

df_p['lasttrip_time'].replace(-9, np.nan, inplace=True)
df_p['firsttrip_time'].replace(-9, np.nan, inplace=True)

df_p.dropna(axis=0, inplace=True)
df_p.reset_index(drop=True, inplace=True)

In [32]:
df_p.shape

(214211, 49)

In [33]:
df_p.to_csv('./data/processed_person_nhts_data.csv')
nhts_df_p = df_p.copy()

## Prepare and normalize the data for training

Only the week days are kept for modeling

In [34]:
# Train the models only on the samples collected with travelday == weekdays
nhts_df_p = nhts_df_p[nhts_df_p['TRAVDAY'] == 'weekday']

In [35]:
# A list of numerical features chosen for modeling
cols_to_norm = ['WORK_EDUCATION',	'SHOP',	'OTHER'	,'AM'	,'PM'	,'MIDDAY'	,'NIGHT',	'firsttrip_time',
                'lasttrip_time'	,	'work_traveltime'	,'shop_traveltime','DWELTIME_mean','work_tripmile',
				'shop_tripmile','TRPMILES_mean',	'TRVL_MIN_mean',	'work_dweltime',
				'shop_dweltime', 'HBO',	'HBSHOP',	'HBW',	'NHB']

# Normalizing numerical features
nhts_df_p[cols_to_norm] = nhts_df_p[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max()-x.min()))

#### Prepare age classification input and output

In [36]:
#defining 5 age ranges
nhts_df_p['R_AGE_CAT'] = np.where((nhts_df_p['R_AGE'] >= 18) & (nhts_df_p['R_AGE'] <= 40), 1,
                         np.where((nhts_df_p['R_AGE'] >= 41) & (nhts_df_p['R_AGE'] <= 50), 2,
                         np.where((nhts_df_p['R_AGE'] >= 51) & (nhts_df_p['R_AGE'] <= 60), 3, 4)))

X_nhts_age = nhts_df_p[cols_to_norm]
y_nhts_age = nhts_df_p['R_AGE_CAT']

#### Prepare income classification input and output

In [37]:
# Contracting income category to 5 classes
nhts_df_p['HHFAMINC'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 1, inplace=True) # 0$-50000$ = 1
nhts_df_p['HHFAMINC'].replace([11, 12, 13, 14, 15, 16, 17], 2, inplace=True) # 50000$-100000$ = 2
nhts_df_p['HHFAMINC'].replace([18], 3, inplace=True) # 100000$ and more = 3

# Copy the dataframe and only drops the nulls of income
nhts_df_p_incom = nhts_df_p.copy()

nhts_df_p_incom['HHFAMINC'].replace([-1,-9,-7,-8], np.nan, inplace=True)
nhts_df_p_incom.dropna(axis=0, inplace=True)
nhts_df_p_incom.reset_index(drop=True, inplace=True)

X_nhts_income = nhts_df_p_incom[cols_to_norm]
y_nhts_income = nhts_df_p_incom['HHFAMINC']

#### Prepare gender classification input and output

In [38]:
# Eliminate people older than 60 for the gender prediction model
nhts_df_p = nhts_df_p[nhts_df_p['R_AGE'] < 60]

X_nhts_sex = nhts_df_p[cols_to_norm]
y_nhts_sex = nhts_df_p['R_SEX']

## 2007/2008 TPB Household Travel Survey

In [191]:
# df is Travel day trip dataframe
df = pd.read_csv('./data/Washington household survey/hts07_TPB_tf.csv')

# df_p is each person's dataframe
df_p = pd.read_csv('./data/Washington household survey/hts07_TPB_pf.csv')

#df_h is for household
df_h = pd.read_csv('./data/Washington household survey/hts07_TPB_hf.csv')

In [192]:
# renaming columns like NHTS data
df.rename(columns={'sampn': 'HOUSEID', 'personid': 'PERSONID','rtripid': 'TDTRPNUM','oact1': 'WHYFROM', 'rdact1': 'WHYTO',
                   'tt': 'TRVL_MIN','dist': 'TRPMILES','begt': 'STRTTIME','endt': 'ENDTIME'}, inplace=True)
df_p.rename(columns={'sampn': 'HOUSEID', 'personid': 'PERSONID','age': 'R_AGE',
                     'gend': 'R_SEX'}, inplace=True)
df_h.rename(columns={'sampn': 'HOUSEID', 'hhsiz': 'HHSIZE','hhveh': 'HHVEHCNT', 'incom':'HHFAMINC'}, inplace=True)

In [193]:
# variables we wish to keep
df = df[
    ['HOUSEID', 'PERSONID','WHYFROM' , 'WHYTO',
     'TRPMILES', 'STRTTIME','ENDTIME','TDTRPNUM',
     'TRVL_MIN', 'pmode']
]
df_p = df_p[
    ['HOUSEID', 'PERSONID', 'R_SEX',
     'R_AGE', 'schol','wkstat', 'hours',
     'relate']
]
df_h = df_h[
    ['HOUSEID', 'HHFAMINC','HHSIZE',
     'HHVEHCNT', 'home_tpb_newtaz', 'hhwrk']
]

In [194]:
# adding values from the household file to the person file
df_p = df_p.join(df_h.set_index('HOUSEID'), on='HOUSEID')

#### family income categories are coded as follow:   
- 1 =Less than $10,000
- 2 =$10,000 - $14,999

- 3 =$15,000 - $29,999

- 4 =$30,000 - $39,999
- 5 =$40,000 - $49,999
- 6 =$50,000 - $59,999

- 7 =$60,000 - $74,999
- 8 =$75,000 - $99,999

- 9 =$100,000 - $124,999
- 10 =$125,000 - $149,999
- 11 =$150,000 - $199,999
- 12 =$200,000 or more

To be consistent with nhts categorization we defined earlier, let's change this:

In [195]:
# changing time to military format, like NHTS
df['STRTTIME'] = df['STRTTIME'].str.replace(':', '')
df['ENDTIME'] = df['ENDTIME'].str.replace(':', '')
df[['STRTTIME','ENDTIME']] = df[['STRTTIME','ENDTIME']].astype(int)

There isn't a columns in TPB data with dweltime values, so let's make one:

In [196]:
# making DWELTIME colum which show the time spent at the destination

def get_minutes(s):
    return s//100 * 60 + s % 100

df['DWELTIME'] = (get_minutes(df.STRTTIME)
                     .groupby([df.HOUSEID, df.PERSONID])
                     .shift(-1)
                     .sub(get_minutes(df.ENDTIME)).fillna(0)
                  )

regrouping purposes as they are:
-    1 = SLEEP/ REST
-   2 = EAT / PREPARE A MEAL AT HOME
-    3 = EAT A MEAL AT WORK
-    4 = EAT A MEAL OUTSIDE HOME OR WORK
-    5 = CARE FOR CHILDREN
-    6 = CHANGE MODE OF TRANSPORTATION
-    7 = PICK UP / DROP OFF SOMEONE OR SOMETHING
-    8 = LOOP TRIP
-    9 = WORK (REGULAR PLACE)
-   10 = WORK AT HOME OR TELECOMMUTE (FOR PAY)
-   11 = WORK AT OTHER LOCATION
-   12 = WORK-RELATED
-   13 = EDUCATION/ SCHOOL-RELATED ACTIVITY
-   14 = STUDY / DO HOMEWORK
-   15 = CHILDCARE / PRESCHOOL
-   16 = SHOP IN STORE
-   17 = SHOP BY PHONE / INTERNET / TV
-   18 = QUICK STOP / DRIVE THRU
-   19 = PERSONAL BUSINESS AT ESTABLISHMENT
-   20 = PERSONAL BUSINESS BY PHONE / INTERNET
-   21 = VISIT/ SOCIALIZE
-   22 = ENTERTAINMENT
-   23 = RECREATION / EXERCISE
-   24 = CIVIC OR RELIGIOUS ACTIVITY
-   25 = MAIL PACKAGE OR LETTER OR OTHER POSTAL
-   26 = OTHER HOUSEHOLD ACTIVITY
-   97 = OTHER

to what they should be: (like we grouped in NHTS preprocess)

In [197]:
# change the items in the given list with 11 and
home_trips = [1, 2, 10, 26, 5]
work_education = [9, 11, 12, 13] # = 2
shop_trips = [16] # = 3
other_trips = [3, 4, 6, 7, 8,
               14, 15, 17, 18, 19,
               20, 21, 22, 23, 24, 25,
               97] # = 4
# add 1 later to the other category

# regrouping purpose of destination classification
df['WHYTO'].replace(home_trips, 1, inplace=True)
df['WHYTO'].replace(other_trips, 3, inplace=True)
df['WHYTO'].replace(work_education, 4, inplace=True)
df['WHYTO'].replace(shop_trips, 2, inplace=True)

# regrouping purpose of origin classification
df['WHYFROM'].replace(home_trips, 1, inplace=True)
df['WHYFROM'].replace(other_trips, 3, inplace=True)
df['WHYFROM'].replace(work_education, 4, inplace=True)
df['WHYFROM'].replace(shop_trips, 2, inplace=True)


In [198]:
# making home-based and non-homebased column
def hb_nhb(whyfrom, whyto):
  if whyfrom != 1 and whyto != 1:
    return 'NHB'
  else:
    if whyfrom == 4 or whyto == 4:
      return 'HBW'
    elif whyfrom == 2 or whyto == 2:
      return 'HBSHOP'
    elif whyfrom == 3 or whyto == 3:
      return 'HBO'

df['TRIPPURP'] = df.apply(lambda row:hb_nhb(row['WHYFROM'], row['WHYTO']), axis=1)

In [199]:
# Changing home purpose to other
df['WHYTO'].replace(1, 3, inplace=True)
df['WHYFROM'].replace(1, 3, inplace=True)

In [200]:
# changing gender classification categories to 0(male) and 1(female)
df_p['R_SEX'].replace(1, 0, inplace=True)
df_p['R_SEX'].replace(2, 1, inplace=True)

In [201]:
#defining 4 time of the day intervals
# 1 = A.M
# 2 = P.M
# 3 = mid_day
# 4 = night

df['TRVL_DAY_TIME'] = np.where((df['STRTTIME'] >= 500) & (df['STRTTIME'] <= 1000), 1, 
                  np.where((df['STRTTIME'] >= 1001) & (df['STRTTIME'] <= 1530), 2,
                  np.where((df['STRTTIME'] >= 1531) & (df['STRTTIME'] <= 2000), 3, 4)))

In [202]:
df['TRPMILES'].replace([-9], np.nan, inplace=True)

In [203]:
#counting the number of home based and non home based trips
counts = df.groupby(["HOUSEID", "PERSONID", "TRIPPURP"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.fillna(0, inplace=True)

In [204]:
# defining 3 columns that shows the number of trips by category
counts = df.groupby(["HOUSEID", "PERSONID", "WHYTO"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"WHYTO{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'WHYTO2': 'WORK_EDUCATION', 'WHYTO3': 'SHOP', 'WHYTO4': 'OTHER'}, inplace=True)

In [205]:
# making the columns for each daytime and merging it to the person dataframe
counts1 = df.groupby(["HOUSEID", "PERSONID", "TRVL_DAY_TIME"]).apply(len).unstack(fill_value=0)
counts1.columns = counts1.columns.map(lambda x: f"STRTTIME{x}")
df_p = df_p.merge(counts1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'STRTTIME1': 'AM', 'STRTTIME2': 'PM', 'STRTTIME3': 'MIDDAY', 'STRTTIME4': 'NIGHT'}, inplace=True)

In [206]:
#this two new columns will show the time the person did their first and last trips, as is obvious in the columns' name
res =(df.sort_values(["HOUSEID","PERSONID"])
    .groupby(["HOUSEID", "PERSONID"], as_index=False)
    .agg({"STRTTIME":"first","ENDTIME":"last"})
    .rename(columns={"STRTTIME":"firsttrip_time","ENDTIME":"lasttrip_time"}))

In [207]:
# this will convert the number to integer numbers in range of 0 to 23
res['firsttrip_time'] = np.where((res['firsttrip_time'] >= 0) & (res['firsttrip_time'] <= 100), 0, 
                  np.where((res['firsttrip_time'] >= 101) & (res['firsttrip_time'] <= 200), 1,
                  np.where((res['firsttrip_time'] >= 201) & (res['firsttrip_time'] <= 300), 2,
                  np.where((res['firsttrip_time'] >= 301) & (res['firsttrip_time'] <= 400), 3,
                  np.where((res['firsttrip_time'] >= 401) & (res['firsttrip_time'] <= 500), 4,
                  np.where((res['firsttrip_time'] >= 501) & (res['firsttrip_time'] <= 600), 5,
                  np.where((res['firsttrip_time'] >= 601) & (res['firsttrip_time'] <= 700), 6,
                  np.where((res['firsttrip_time'] >= 701) & (res['firsttrip_time'] <= 800), 7,
                  np.where((res['firsttrip_time'] >= 801) & (res['firsttrip_time'] <= 900), 8,
                  np.where((res['firsttrip_time'] >= 901) & (res['firsttrip_time'] <= 1000), 9,
                  np.where((res['firsttrip_time'] >= 1001) & (res['firsttrip_time'] <= 1100), 10,
                  np.where((res['firsttrip_time'] >= 1101) & (res['firsttrip_time'] <= 1200), 11,
                  np.where((res['firsttrip_time'] >= 1201) & (res['firsttrip_time'] <= 1300), 12, 
                  np.where((res['firsttrip_time'] >= 1301) & (res['firsttrip_time'] <= 1400), 13,
                  np.where((res['firsttrip_time'] >= 1401) & (res['firsttrip_time'] <= 1500), 14,
                  np.where((res['firsttrip_time'] >= 1501) & (res['firsttrip_time'] <= 1600), 15,
                  np.where((res['firsttrip_time'] >= 1601) & (res['firsttrip_time'] <= 1700), 16,
                  np.where((res['firsttrip_time'] >= 1701) & (res['firsttrip_time'] <= 1800), 17,
                  np.where((res['firsttrip_time'] >= 1801) & (res['firsttrip_time'] <= 1900), 18,
                  np.where((res['firsttrip_time'] >= 1901) & (res['firsttrip_time'] <= 2000), 19,
                  np.where((res['firsttrip_time'] >= 2001) & (res['firsttrip_time'] <= 2100), 20,
                  np.where((res['firsttrip_time'] >= 2101) & (res['firsttrip_time'] <= 2200), 21,
                  np.where((res['firsttrip_time'] >= 2201) & (res['firsttrip_time'] <= 2300), 22,
                  np.where((res['firsttrip_time'] >= 2301) & (res['firsttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))

res['lasttrip_time'] = np.where((res['lasttrip_time'] >= 0) & (res['lasttrip_time'] <= 100), 0, 
                  np.where((res['lasttrip_time'] >= 101) & (res['lasttrip_time'] <= 200), 1,
                  np.where((res['lasttrip_time'] >= 201) & (res['lasttrip_time'] <= 300), 2,
                  np.where((res['lasttrip_time'] >= 301) & (res['lasttrip_time'] <= 400), 3,
                  np.where((res['lasttrip_time'] >= 401) & (res['lasttrip_time'] <= 500), 4,
                  np.where((res['lasttrip_time'] >= 501) & (res['lasttrip_time'] <= 600), 5,
                  np.where((res['lasttrip_time'] >= 601) & (res['lasttrip_time'] <= 700), 6,
                  np.where((res['lasttrip_time'] >= 701) & (res['lasttrip_time'] <= 800), 7,
                  np.where((res['lasttrip_time'] >= 801) & (res['lasttrip_time'] <= 900), 8,
                  np.where((res['lasttrip_time'] >= 901) & (res['lasttrip_time'] <= 1000), 9,
                  np.where((res['lasttrip_time'] >= 1001) & (res['lasttrip_time'] <= 1100), 10,
                  np.where((res['lasttrip_time'] >= 1101) & (res['lasttrip_time'] <= 1200), 11,
                  np.where((res['lasttrip_time'] >= 1201) & (res['lasttrip_time'] <= 1300), 12, 
                  np.where((res['lasttrip_time'] >= 1301) & (res['lasttrip_time'] <= 1400), 13,
                  np.where((res['lasttrip_time'] >= 1401) & (res['lasttrip_time'] <= 1500), 14,
                  np.where((res['lasttrip_time'] >= 1501) & (res['lasttrip_time'] <= 1600), 15,
                  np.where((res['lasttrip_time'] >= 1601) & (res['lasttrip_time'] <= 1700), 16,
                  np.where((res['lasttrip_time'] >= 1701) & (res['lasttrip_time'] <= 1800), 17,
                  np.where((res['lasttrip_time'] >= 1801) & (res['lasttrip_time'] <= 1900), 18,
                  np.where((res['lasttrip_time'] >= 1901) & (res['lasttrip_time'] <= 2000), 19,
                  np.where((res['lasttrip_time'] >= 2001) & (res['lasttrip_time'] <= 2100), 20,
                  np.where((res['lasttrip_time'] >= 2101) & (res['lasttrip_time'] <= 2200), 21,
                  np.where((res['lasttrip_time'] >= 2201) & (res['lasttrip_time'] <= 2300), 22,
                  np.where((res['lasttrip_time'] >= 2301) & (res['lasttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))
x = res.groupby(['HOUSEID','PERSONID']).sum()# to prevent an error
df_p = df_p.merge(x, how="left", left_on=['HOUSEID', 'PERSONID'], right_index=True)

In [208]:
# this makes 2 columns of mean travel time for work and shop trips 
new_df1=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRVL_MIN')
new_df1.fillna(0, inplace=True)

df_p = df_p.merge(new_df1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_traveltime', 3:'shop_traveltime'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)

In [209]:
#sum of all lenght traveled by the person
new_df2=df.groupby(['HOUSEID','PERSONID'] )['TRPMILES'].sum()
df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_sum'}, inplace=True)

#sum of all the time the person was traveling
new_df3=df.groupby(['HOUSEID','PERSONID'] )['TRVL_MIN'].sum()
df_p = df_p.merge(new_df3, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_sum'}, inplace=True)

In [210]:
df['TRPMILES'] /= df['TRPMILES'].max()

new_df=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRPMILES')
new_df.fillna(0, inplace=True)

df_p = df_p.merge(new_df, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_tripmile', 3: 'shop_tripmile'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)

In [211]:
#trip miles for each person, normalized by state and urban-rural classification
s=df.groupby(['HOUSEID',  'PERSONID'])['TRPMILES'].mean()
df_p = df_p.merge(s, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_mean'}, inplace=True)

#mean traveltime of all trips
s1=df.groupby(['HOUSEID',  'PERSONID'])['TRVL_MIN'].mean()
df_p = df_p.merge(s1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_mean'}, inplace=True)

#mean of the time the person is not traveling
s2=df.groupby(['HOUSEID',  'PERSONID'])['DWELTIME'].mean()
df_p = df_p.merge(s2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'DWELTIME': 'DWELTIME_mean'}, inplace=True)

In [212]:
# mean time spent  for shopping and work 
new_df2=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='DWELTIME')
new_df2.fillna(0, inplace=True)

df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_dweltime', 3:'shop_dweltime'}, inplace=True)
df_p.drop([4], axis=1, inplace=True)

In [213]:
# Detting under 18 people as np.nan
list_x_under_18 = [i for i in range(0,18)]
df_p['R_AGE'].replace(list_x_under_18, np.nan, inplace=True)

In [214]:
df_p.dropna(axis=0, inplace=True)
df_p.reset_index(drop=True, inplace=True)

In [215]:
tpb_df_p = df_p.copy()

## Prepare and normalize the data for training

In [262]:
# normalizing data
cols_to_norm = ['WORK_EDUCATION',	'SHOP',	'OTHER'	,'AM'	,'PM'	,'MIDDAY'	,'NIGHT',	'firsttrip_time',
                'lasttrip_time'	,	'work_traveltime'	,'shop_traveltime','DWELTIME_mean','work_tripmile',
				'shop_tripmile','TRPMILES_mean',	'TRVL_MIN_mean',	'work_dweltime',
				'shop_dweltime', 'HBO',	'HBSHOP',	'HBW',	'NHB']
                  
tpb_df_p[cols_to_norm] = tpb_df_p[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max()-x.min())) 

#### Prepare age classification input and output


Age ranges:
- 0-18 = 1
- 19-32 = 2
- 33-46 = 3
- 47-60 = 4
- 60 and more = 5

In [263]:
#defining 5 age ranges
tpb_df_p['R_AGE_CAT'] = np.where((tpb_df_p['R_AGE'] >= 18) & (tpb_df_p['R_AGE'] <= 40), 1,
                         np.where((tpb_df_p['R_AGE'] >= 41) & (tpb_df_p['R_AGE'] <= 50), 2,
                         np.where((tpb_df_p['R_AGE'] >= 51) & (tpb_df_p['R_AGE'] <= 60), 3,                                                                 
                                  4)))                            

X_tpb_age = tpb_df_p[cols_to_norm]
y_tpb_age = tpb_df_p['R_AGE_CAT']

#### Prepare income classification input and output

In [264]:
#contracting income category to 5 classes
tpb_df_p['HHFAMINC'].replace([1, 2, 3, 4, 5], 1, inplace=True) # 0$-50000$ = 1
tpb_df_p['HHFAMINC'].replace([6, 7, 8], 2, inplace=True) # 50000$-100000$ = 2
tpb_df_p['HHFAMINC'].replace([9, 10, 11, 12], 3, inplace=True) # 100000$ and more  = 3

X_tpb_income = tpb_df_p[cols_to_norm]
y_tpb_income = tpb_df_p['HHFAMINC']

#### Prepare gender classification input and output

In [265]:
tpb_df_p = tpb_df_p[tpb_df_p['R_AGE'] < 60]
X_tpb_sex = tpb_df_p[cols_to_norm]
y_tpb_sex = tpb_df_p['R_SEX']

# Model training

In [82]:
def catboost_model(X_train, X_test, y_train, y_test,
                    learning_rate=0.01,bagging_temperature=0.95,
                    l2_leaf_reg=10, iterations=100000, 
                    early_stopping_rounds=1000, border_count=128,
                    depth=6, random_strength=2.87, task='clasification'):
                    
    # Set parameters
    PARAMS_CATBOOST = dict()
    PARAMS_CATBOOST['learning_rate']= learning_rate
    PARAMS_CATBOOST['bagging_temperature']= bagging_temperature
    PARAMS_CATBOOST['l2_leaf_reg'] = l2_leaf_reg # lambda, default 3, S: 300
    PARAMS_CATBOOST['iterations']= iterations
    PARAMS_CATBOOST['early_stopping_rounds']=early_stopping_rounds
    PARAMS_CATBOOST['border_count']= border_count
    PARAMS_CATBOOST['depth']= depth
    PARAMS_CATBOOST['random_strength']= random_strength
    PARAMS_CATBOOST['use_best_model']= True
    # PARAMS_CATBOOST_REGRESSOR['logging_level'] = 'Silent'

    cat_features=[]
    if task =='regression':
        boost_model = CatBoostRegressor(**PARAMS_CATBOOST)
    elif task == 'clasification':
        boost_model = CatBoostClassifier(**PARAMS_CATBOOST)


    train_dataset = Pool(data=X_train,
                        label=y_train,
                        cat_features=cat_features)
    val_dataset = Pool(data=X_test,
                        label=y_test,
                        cat_features=cat_features)

    boost_model.fit(train_dataset,use_best_model=True, eval_set=val_dataset)
    
    return boost_model

In [72]:
def nn_model(X_train, X_test, y_train, y_test,
            hidden_act='relu', task='clasification',
            epochs=100, batch_size=128, verbose=0):

    optimizer = Adam(learning_rate=0.001)
    loss_fn = keras.losses.CategoricalCrossentropy()

    inputs = keras.Input(X_train.shape[1],)
    x = layers.Dense(128, activation=hidden_act)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation=hidden_act)(x)
    x = keras.layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(X_train.shape[1], activation=hidden_act)(x)
    x = keras.layers.BatchNormalization()(x)
    x = layers.Add()([x, inputs])
    x = layers.Dense(X_train.shape[1], activation=hidden_act)(x)
    x = keras.layers.BatchNormalization()(x)
    x = layers.Activation(hidden_act)(x)
    x = layers.Add()([x, inputs])

    if task == 'regression':
        outputs = layers.Dense(1, activation='sigmoid')(x)
        model = keras.Model(inputs, outputs)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
        
    elif task == 'clasification':
        y_train = (pd.get_dummies(y_train)).values
        y_test = (pd.get_dummies(y_test)).values
        class_number = y_train.shape[1]
        outputs = layers.Dense(class_number, activation='softmax')(x)
        model = keras.Model(inputs, outputs)
        model.compile(loss=loss_fn, optimizer=optimizer, metrics=['acc'])
        
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=verbose)
    
    return model

In [90]:
def evaluate(model, X_test, y_test, mode='normal', print_results=False, average='weighted'):
    
    y_pred = model.predict(X_test)
    if mode == 'regression':
        y_pred = np.array([1 if i > 0.5 else 0 for i in y_pred])
    elif mode == 'onehot':
        # convert onehot encoded output to integer
        y_pred = np.argmax(y_pred, axis=1)
    else:
        pass

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average=average)
    recall = recall_score(y_test, y_pred,average=average)
    f1 = f1_score(y_test, y_pred,average=average)
    confusion = confusion_matrix(y_test, y_pred)
    cohen_kappa = cohen_kappa_score(y_test, y_pred)

    if print_results == True:
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('F1 score:', f1)
        print('Confusion matrix:', confusion)
        print('Cohen kappa score', cohen_kappa)

    return accuracy, precision, recall, f1, confusion, cohen_kappa
    

In [88]:
def general_ml_models(x_train, x_test, y_train, y_test,
                     kernel_type="rbf", max_iteration=10000,
                        n_neig=10, algorithm_type="auto", n_est=100, boot_strap=True,
                        max_feat="sqrt"):
    # defime models
    knn = KNeighborsClassifier(n_neighbors=n_neig, weights="distance", algorithm=algorithm_type)
    rf = RandomForestClassifier(n_estimators =n_est, bootstrap=boot_strap, max_features=max_feat)
    svm = SVC(kernel=kernel_type, max_iter=max_iteration, gamma="auto")
    adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=n_est, learning_rate=0.1)
    Gb = GradientBoostingClassifier(random_state=0)
    dt = DecisionTreeClassifier()
    lr = LogisticRegression()
    lgbm = LGBMClassifier()

    # fit models
    knn.fit(x_train, y_train)
    rf.fit(x_train, y_train)
    svm.fit(x_train, y_train)
    adaboost.fit(x_train, y_train)
    Gb.fit(x_train, y_train)
    dt.fit(x_train, y_train)
    #lr.fit(x_train, y_train)
    lgbm.fit(x_train, y_train)

    # predict 
    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(knn, x_test, y_test)
    print('The accuracy score for knn is:\n', accuracy)
    
    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(rf, x_test, y_test)
    print('The accuracy score for rf is:\n', accuracy)
    
    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(svm, x_test, y_test)
    print('The accuracy score for svm is:\n', accuracy)

    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(adaboost, x_test, y_test)
    print('The accuracy score for adaboost is:\n', accuracy)

    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(Gb, x_test, y_test)
    print('The accuracy score for gradient boost is:\n', accuracy)

    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(dt, x_test, y_test)
    print('The accuracy score for decision tree is:\n', accuracy)

    '''accuracy_score, precision_score, recall_score, \
    f1_score, confusion_matrix, cohen_kappa_score = evaluate(lr, x_test, y_test)
    print('The accuracy score for logistic regression is:\n', accuracy_score)'''

    accuracy, precision, recall, \
    f1, confusion, cohen_kappa = evaluate(lgbm, x_test, y_test)
    print('The  accuracy score for lightgbm is:\n', accuracy)

#### Split into train and test

In [75]:
from sklearn.model_selection import train_test_split

# NHTS
X_train_nhts_age, X_test_nhts_age, y_train_nhts_age, y_test_nhts_age =train_test_split(X_nhts_age, y_nhts_age, test_size=0.3, random_state=432)
# TPB
X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age =train_test_split(X_tpb_age, y_tpb_age, test_size=0.3, random_state=432)

# NHTS
X_train_nhts_income, X_test_nhts_income, y_train_nhts_income, y_test_nhts_income =train_test_split(X_nhts_income, y_nhts_income, test_size=0.3, random_state=432)
# TPB
X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income =train_test_split(X_tpb_income, y_tpb_income, test_size=0.3, random_state=432)

# NHTS
X_train_nhts_sex, X_test_nhts_sex, y_train_nhts_sex, y_test_nhts_sex =train_test_split(X_nhts_sex, y_nhts_sex, test_size=0.3, random_state=432)
# TPB
X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex =train_test_split(X_tpb_sex, y_tpb_sex, test_size=0.3, random_state=432)

### Age

In [ ]:
#nueral network

#nhts
nn_nhts_age = nn_model(X_train_nhts_age, X_test_nhts_age, y_train_nhts_age, y_test_nhts_age, verbose=1)
#tpb

nn_tpb_age = nn_model(X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age, verbose=1)

In [ ]:
# Catboost

#nhts
cat_nhts_age = catboost_model(X_train_nhts_age, X_test_nhts_age, y_train_nhts_age, y_test_nhts_age)
#tpb
cat_tpb_age = catboost_model(X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age)

In [ ]:
# other models
other_tpb_age = general_ml_models(X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age)

### Income

In [ ]:
#nueral network

#nhts
nn_nhts_income = nn_model(X_train_nhts_income, X_test_nhts_income, y_train_nhts_income, y_test_nhts_income, verbose=1)
#tpb
nn_tpb_income = nn_model(X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income, verbose=1)

In [ ]:
# Catboost

#nhts
cat_nhts_income = catboost_model(X_train_nhts_income, X_test_nhts_income, y_train_nhts_income, y_test_nhts_income)
#tpb
cat_tpb_income = catboost_model(X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income)

In [ ]:
# other models
other_tpb_income = general_ml_models(X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income)

### Gender

In [ ]:
#nueral network

#nhts
nn_nhts_gender = nn_model(X_train_nhts_sex, X_test_nhts_sex, y_train_nhts_sex, y_test_nhts_sex, task='regression', verbose=1)
#tpb
nn_tpb_gender = nn_model(X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex, task='regression', verbose=1)

In [ ]:
# Catboost

#nhts
cat_nhts_gender = catboost_model(X_train_nhts_sex, X_test_nhts_sex, y_train_nhts_sex, y_test_nhts_sex, task='regression')
#tpb
cat_tpb_gender = catboost_model(X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex, task='regression')

In [ ]:
# Other models
other_tpb_gender = general_ml_models(X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex)

## Evaluation

### Age

In [ ]:
# nn nhts
evaluate(nn_nhts_age, X_test_nhts_age, y_test_nhts_age, mode='onehot', print_results=True)

In [ ]:
# cat nhts
evaluate(cat_nhts_age, X_test_nhts_age, y_test_nhts_age, mode='normal', print_results=True)

In [ ]:
# nn tpb
evaluate(nn_tpb_age, X_test_tpb_age, y_test_tpb_age, mode='onehot', print_results=True)

In [ ]:
# cat tpb
evaluate(cat_tpb_age, X_test_tpb_age, y_test_tpb_age, mode='normal', print_results=True)

### Income

In [ ]:
# nn nhts
evaluate(nn_nhts_income, X_test_nhts_income, y_test_nhts_income, mode='onehot', print_results=True)

In [ ]:
# cat nhts
evaluate(cat_nhts_income, X_test_nhts_income, y_test_nhts_income, mode='normal', print_results=True)

In [ ]:
# nn tpb
evaluate(nn_tpb_income, X_test_tpb_income, y_test_tpb_income, mode='onehot', print_results=True)

In [ ]:
# cat tpb
evaluate(cat_tpb_income, X_test_tpb_income, y_test_tpb_income, mode='normal', print_results=True)

### Gender

In [ ]:
# nn nhts
evaluate(nn_nhts_gender, X_test_nhts_sex, y_test_nhts_sex, mode='regression', print_results=True)

In [ ]:
# cat nhts
evaluate(cat_nhts_gender, X_test_nhts_sex, y_test_nhts_sex, mode='regression', print_results=True)

In [ ]:
# nn tpb
evaluate(nn_tpb_gender, X_test_tpb_sex, y_test_tpb_sex, mode='regression', print_results=True)

In [ ]:
# cat tpb
evaluate(cat_tpb_gender,X_test_tpb_sex, y_test_tpb_sex, mode='regression', print_results=True)

# Post modeling -> Bayesian inference for income

In this section synthesized data is used to improve the probalities of income and age predictions based on Bayesian updating 

In [ ]:
syn_df = pd.read_csv('./data/New_Households_base_year2.csv.flattened.csv',index_col=0)

syn_df.shape()

In [ ]:
syn_df.head()

In [112]:
 #setting under 18 people as np.nan
syn_df['AGE'].replace(list_under18, np.nan, inplace=True)
syn_df.dropna(axis=0, inplace=True)
syn_df.reset_index(drop=True, inplace=True)

In [113]:
#defining 5 age ranges
syn_df['INCOME'] = np.where((syn_df['INCOME'] >= 0) & (syn_df['INCOME'] <= 50000), 1,
                         np.where((syn_df['INCOME'] >= 50001) & (syn_df['INCOME'] <= 100000), 2,                
                            3))

In [114]:
#defining 5 age ranges
syn_df['AGE'] = np.where((syn_df['AGE'] >= 18) & (syn_df['AGE'] <= 40), 1,
                         np.where((syn_df['AGE'] >= 41) & (syn_df['AGE'] <= 50), 2,
                         np.where((syn_df['AGE'] >= 51) & (syn_df['AGE'] <= 60), 3,                 
                            4)))

In [115]:
# changing gender classification categories to 0(male) and 1(female)
syn_df['GENDER'].replace(1, 0, inplace=True)
syn_df['GENDER'].replace(2, 1, inplace=True)

In [ ]:
#grouby counts and calculate the prevalent distribution percentage for each location
counts_income = syn_df.groupby(["LOCATION", "INCOME"]).apply(len).unstack(fill_value=0)
counts_income['sum'] = counts_income[1]+counts_income[2]+counts_income[3]
counts_income[1] = counts_income[1]/counts_income['sum']
counts_income[2] = counts_income[2]/counts_income['sum']
counts_income[3] = counts_income[3]/counts_income['sum']
del counts_income['sum']

In [ ]:
#grouby counts and calculate the prevalent distribution percentage for each location
counts_age = syn_df.groupby(["LOCATION", "AGE"]).apply(len).unstack(fill_value=0)
counts_age['sum'] = counts_age[1]+counts_age[2]+counts_age[3]+counts_age[4]
counts_age[1] = counts_age[1]/counts_age['sum']
counts_age[2] = counts_age[2]/counts_age['sum']
counts_age[3] = counts_age[3]/counts_age['sum']
counts_age[4] = counts_age[4]/counts_age['sum']
del counts_age['sum']

In [ ]:
#grouby counts and calculate the prevalent distribution percentage for each location
counts_sex = syn_df.groupby(["LOCATION", "GENDER"]).apply(len).unstack(fill_value=0)
counts_sex['sum'] = counts_sex[0]+counts_sex[1]
counts_sex[0] = counts_sex[0]/counts_sex['sum']
counts_sex[1] = counts_sex[1]/counts_sex['sum']
del counts_sex['sum']

In [172]:
#making possiblity dictionaries
income_likelihood_dict = counts_income.T.to_dict('list')
age_likelihood_dict = counts_age.T.to_dict('list')
sex_likelihood_dict = counts_sex.T.to_dict('list')

Now predict the income probabilities from the model and update them based on TAZ

In [123]:
# predict labels for all the data
y_pred_tpb_income = cat_tpb_income.predict(X_tpb_income)

# predict probabilies for all the data
y_pred_proba_tpb_income = cat_tpb_income.predict_proba(X_tpb_income)

In [217]:
# convert numpy array to dataframe 
df = pd.DataFrame(y_pred_proba_tpb_income, columns =['prob1', 'prob2', 'prob3'])

#assign location to the probablity dataframe
df['Location'] = tpb_df_p['home_tpb_newtaz']

#assign y_true to the dataframe
df['y_true'] = y_tpb_income

#predicted y
df['y_pred'] = y_pred_tpb_income

In [219]:
#formula bays. update probability
def bays_update(prob1, prob2, prob3, location):
    if location in income_likelihood_dict:
        list_prob = income_likelihood_dict[location]
        liky_1 = list_prob[0]# likelyhood = liky
        liky_2 = list_prob[1]
        liky_3 = list_prob[2]
        #soorat kasr
        numerator1 = (liky_1*prob1) 
        numerator2 = (liky_2*prob2)
        numerator3 = (liky_3*prob3)
        #makhraj kasr
        evidence = numerator1+numerator2+numerator3
        posterior1 = numerator1/evidence
        posterior2 = numerator2/evidence
        posterior3 = numerator3/evidence
        return pd.Series([posterior1,posterior2,posterior3])
    else:
        return pd.Series([prob1,prob2,prob3])

In [220]:
df['updated_prob1'] = ''
df['updated_prob2'] = ''
df['updated_prob3'] = ''
df[['updated_prob1', 'updated_prob2', 'updated_prob3']] = (df.apply(lambda row:bays_update(row['prob1'], row['prob2'], row['prob3'], row['Location']),axis=1))

In [221]:
def max_value(prob1,prob2,prob3):
  maximum = max(prob1,prob2,prob3)
  if maximum == prob1:
    return 1
  elif maximum == prob2:
    return 2
  elif maximum == prob3:
    return 3

In [222]:
df['updated_pred'] = (df.apply(lambda row:max_value(row['updated_prob1'], row['updated_prob2'], row['updated_prob3']),axis=1))

In [ ]:
df

In [ ]:
accuracy_score(df['y_true'], df['updated_pred'])

# Experiment

In [ ]:
# both data are prepared in the same manner so these experiments are possible to conduct
X_nhts_age.columns == X_tpb_age.columns

## 1- Train on NHTS and validate on TPB

In [ ]:
ex_age_model = nn_model(X_nhts_age, X_tpb_age, y_nhts_age, y_tpb_age, epochs=10, verbose=1)

In [ ]:
ex_income_model = nn_model(X_nhts_income, X_tpb_income, y_nhts_income, y_tpb_income, epochs=10, verbose=1)

In [ ]:
ex_gender_model = nn_model(X_nhts_sex, X_tpb_sex, y_nhts_sex, y_tpb_sex, task='regression', epochs=10, verbose=1)

## 2- Use the NHTS model to improve TPB model

In [266]:

# predict probabilities

y_tpb_on_nhts_age = cat_nhts_age.predict_proba(X_tpb_age)
y_tpb_on_nhts_income = cat_nhts_income.predict_proba(X_tpb_income)
y_tpb_on_nhts_gender = cat_nhts_gender.predict(X_tpb_sex)

# change the numpy array to dataframe
y_tpb_on_nhts_age = pd.DataFrame(y_tpb_on_nhts_age)
y_tpb_on_nhts_income = pd.DataFrame(y_tpb_on_nhts_income)
y_tpb_on_nhts_gender = pd.DataFrame(y_tpb_on_nhts_gender)

# reset index and contacenate
X_tpb_age.reset_index(drop=True, inplace=True)
X_tpb_income.reset_index(drop=True, inplace=True)
X_tpb_sex.reset_index(drop=True, inplace=True)

X_tpb_age = pd.concat([X_tpb_age, y_tpb_on_nhts_age], axis=1)
X_tpb_income = pd.concat([X_tpb_income, y_tpb_on_nhts_income], axis=1)
X_tpb_sex = pd.concat([X_tpb_sex, y_tpb_on_nhts_gender], axis=1)


In [271]:
# split the new data for training

X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age = train_test_split(X_tpb_age, y_tpb_age, test_size=0.3, random_state=432)
X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income = train_test_split(X_tpb_income, y_tpb_income, test_size=0.3, random_state=432)
X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex = train_test_split(X_tpb_sex, y_tpb_sex, test_size=0.3, random_state=432)

In [ ]:
cat_tpb_age_exp = catboost_model(X_train_tpb_age, X_test_tpb_age, y_train_tpb_age, y_test_tpb_age)
evaluate(cat_tpb_age_exp, X_test_tpb_age, y_test_tpb_age, print_results=True)

In [ ]:
cat_tpb_income_exp = catboost_model(X_train_tpb_income, X_test_tpb_income, y_train_tpb_income, y_test_tpb_income)
evaluate(cat_tpb_income_exp, X_test_tpb_income, y_test_tpb_income, print_results=True)

In [ ]:
cat_tpb_gender_exp = catboost_model(X_train_tpb_sex, X_test_tpb_sex, y_train_tpb_sex, y_test_tpb_sex, task='regression')
evaluate(cat_tpb_gender_exp, X_test_tpb_sex, y_test_tpb_sex, mode='regression', print_results=True)